In [1]:
import numpy as np
import pickle
import anndata as ad
from sklearn.model_selection import train_test_split
import warnings

from data.data_process import data_process
from model.deconv_model import MBdeconv
from model.utils import *

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# 在使用GPU时，还可以设置以下代码来确保结果的一致性
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
warnings.filterwarnings("ignore")

# data

In [2]:
# Read data
data_file = 'data/bone_marrow_mb/bone_marrow.h5ad'
data_h5ad = ad.read_h5ad(data_file)

In [3]:
# Merge cell subtypes
data_h5ad.obs['CellType'] = data_h5ad.obs['CellType'].replace({'HSC (catulin+)': 'HSC', 'HSC (catulin-)': 'HSC'})

In [4]:
type_list = ['Erythroid', 'T', 'B', 'GMP', 'Myeloid']
noise = ['HSC']

In [5]:
# Noise cell extraction
print("Loading data...")
# extract noise
if noise:
    data_h5ad_noise = data_h5ad[data_h5ad.obs['CellType'].isin(noise)]
    data_h5ad_noise.obs.reset_index(drop=True, inplace=True)
# extract selected cells 
data_h5ad = data_h5ad[data_h5ad.obs['CellType'].isin(type_list)]
data_h5ad.obs.reset_index(drop=True, inplace=True)
print('selected cells:', data_h5ad)

Loading data...
selected cells: View of AnnData object with n_obs × n_vars = 653 × 107
    obs: 'CellType'


In [6]:
# Cell separation used for training and testing.
train_idx = []
test_idx = []

# Selected cells split into train and test datasets
for cell_type in data_h5ad.obs['CellType'].unique():
    current_idx = data_h5ad.obs[data_h5ad.obs['CellType'] == cell_type].index.tolist()
    train_i, test_i = train_test_split(current_idx, test_size=0.5, random_state=42)
    train_idx.extend(train_i)
    test_idx.extend(test_i)

print("Selected cells split into train and test datasets.")
train_data = data_h5ad[train_idx]
test_data = data_h5ad[test_idx]

Selected cells split into train and test datasets.


In [7]:
dp = data_process(type_list, tissue_name='bone_marrow_mb', 
                  test_sample_num=1000, sample_size=30, num_artificial_cells=30)

In [8]:
dp.fit(train_data, test_data, data_h5ad_noise)

Generating artificial cells...
Generating train pseudo_bulk samples...


train Samples: 100%|██████████| 6000/6000 [00:15<00:00, 397.26it/s]


Generating test pseudo_bulk samples...


test Samples: 100%|██████████| 1000/1000 [00:02<00:00, 374.23it/s]


The data processing is complete


In [9]:
with open(f'data/bone_marrow_mb/bone_marrow_mb{len(type_list)}cell.pkl', 'rb') as f:
    train = pickle.load(f)
    test = pickle.load(f)
    test_with_noise = pickle.load(f)

In [10]:
train_x_sim, train_with_noise_1, train_with_noise_2, train_y = train
test_x_sim, test_y = test

valid_size = 1000  

# 切片操作  
valid_x_sim = train_x_sim[:valid_size]  
valid_with_noise_1 = train_with_noise_1[:valid_size]  
valid_with_noise_2 = train_with_noise_2[:valid_size]  
valid_y = train_y[:valid_size]  

train_x_sim = train_x_sim[valid_size:]  
train_with_noise_1 = train_with_noise_1[valid_size:]  
train_with_noise_2 = train_with_noise_2[valid_size:]  
train_y = train_y[valid_size:]  

test_dataset = TestCustomDataset(test_x_sim, test_y)
valid_dataset = TestCustomDataset(valid_x_sim, valid_y)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

train_dataset = TrainCustomDataset(train_x_sim, train_with_noise_1, train_with_noise_2, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# model

In [11]:
num_MB = 107
feat_map_w = 256
feat_map_h = 10
num_cell_type = len(type_list)
epoches = 500
patience = 50
Alpha = 1
Beta = 1
model_save_name = 'bone_marrow_mb'

In [12]:
model = MBdeconv(num_MB, feat_map_w, feat_map_h, num_cell_type, epoches, Alpha, Beta, train_dataloader, valid_dataloader)

In [13]:
device = torch.device('cuda')
if model.gpu_available:
    model = model.to(model.gpu)

In [14]:
# Skip training Stage 2 and directly train Stage 3
loss1_list, loss2_list, nce_loss_list, total_list = model.train_model(model_save_name, True, patience)


===== Starting Training (Total Epochs: 500) =====
Patience for early stopping: 50 epochs



Epoch 1/500 | Loss: 4.1419 Curr: 4.0281: |          | 151/? batches    


[Ep 1] 1.5s | Loss: 4.1132 (L1: 0.0133, L2: 0.0133, NCE: 8.1471) | Test: RMSE=0.1112, MAE=0.0912
  ★ Best RMSE! Model saved at bone_marrow_mb


Epoch 2/500 | Loss: 4.1234 Curr: 4.0945: |          | 151/? batches    


[Ep 2] 2.8s | Loss: 4.0915 (L1: 0.0130, L2: 0.0130, NCE: 8.1071) | Test: RMSE=0.1073, MAE=0.0877
  ★ Best RMSE! Model saved at bone_marrow_mb


...

[Ep 427] 516.3s | Loss: 3.0251 (L1: 0.0028, L2: 0.0028, NCE: 6.0336) | Test: RMSE=0.0499, MAE=0.0389
  ↯ No improvement (48/50)


Epoch 428/500 | Loss: 3.0204 Curr: 2.9945: |          | 151/? batches    


[Ep 428] 517.5s | Loss: 3.0118 (L1: 0.0027, L2: 0.0027, NCE: 6.0075) | Test: RMSE=0.0504, MAE=0.0398
  ↯ No improvement (49/50)


Epoch 429/500 | Loss: 3.0112 Curr: 3.0040: |          | 151/? batches    

[Ep 429] 518.8s | Loss: 2.9895 (L1: 0.0026, L2: 0.0026, NCE: 5.9631) | Test: RMSE=0.0664, MAE=0.0509
  ↯ No improvement (50/50)

Early stopping triggered at epoch 429!
Best RMSE achieved: 0.0497


===== Training Complete! =====
Total training time: 518.8 seconds
Best epoch: 379 with RMSE = 0.0497
Final results: RMSE=0.0664, MAE=0.0509



In [15]:
model_test = MBdeconv(num_MB, feat_map_w, feat_map_h, num_cell_type, epoches, Alpha, Beta, train_dataloader, test_dataloader)
model_test.load_state_dict(torch.load('save_models/107/bone_marrow_mb.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_test.to(device)
model_test.eval()
CCC, RMSE, Corr, pred, gt = predict(test_dataloader, type_list, model_test, True)
CCC, RMSE, Corr

(0.8080067244154101, 0.06551918271747084, 0.8163028002015478)